## Preparations
* TensorFlow version == 1.x required
* Gast version == 0.2.x, TensorFlow-Gan version == 1.x required

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import numpy as np
import tensorflow as tf
from scipy.stats import norm
import matplotlib.pyplot as plt
tfd = tf.contrib.distributions

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
import utils
import nets
import flow_layers as fl

In [ ]:
print(tf.__version__)

In [ ]:
config = tf.ConfigProto()
config.allow_soft_placement=True
config.gpu_options.allow_growth = True
tf.set_random_seed(0)
sess = tf.InteractiveSession(config=config)

## Main Part

### Import Dataset

In [ ]:
batch_size = 1
image_size = 128
x_valid_samples = utils.create_tfrecord_dataset_iterator(
    "test_1.tfrecords", batch_size=batch_size, image_size=image_size
)

### Plot Input

In [ ]:
x_exampled = x_valid_samples.eval()
plt.imshow(x_exampled[0,:,:,:])

### Build Encoder Forward Flow

In [ ]:
nn_template_fn = nets.OpenAITemplate(
    width=128
)

In [ ]:
layers_encoder, actnorm_layers_encoder = nets.create_simple_flow(
    num_steps=32, 
    num_scales=5, 
    template_fn=nn_template_fn
)

image = x_valid_samples
flow_encoder = fl.InputLayer(image)
model_flow_encoder = fl.ChainLayer(layers_encoder)
encoder_output = model_flow_encoder(flow_encoder, forward=True)

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, "t1-2-saves/steps.ckpt")

In [ ]:
a, b, c = encoder_output

### Save Outputs

In [ ]:
from tensorflow.python.ops import io_ops
save_op_1 = io_ops._save(filename="aux-saves/a.ckpt", tensor_names=["a"],
                       tensors=[a])
save_op_2 = io_ops._save(filename="aux-saves/b.ckpt", tensor_names=["b"],
                       tensors=[b])
save_op_3 = io_ops._save(filename="aux-saves/c.ckpt", tensor_names=["c"],
                       tensors=[c])

In [ ]:
sess.run(save_op_1) 
sess.run(save_op_2) 
sess.run(save_op_3) 